In [1]:
from importlib import reload
import environments.ControlledRangeVariance
import MLE.MLE 

reload(environments.ControlledRangeVariance)
reload(MLE.MLE)

def batchtoonline(samples, seed=45):
    import numpy as np
    
    state = np.random.RandomState(seed)
    
    n = sum(c for c, w, r in samples)
    while n > 0:
        p = np.array([ c for c, w, r in samples ], dtype='float64') / n
        what = state.choice(len(samples), p=p)
        c = min(samples[what][0], 1)
        yield (c, samples[what][1], samples[what][2])
        samples[what] = (samples[what][0] - c, samples[what][1], samples[what][2]) 
        n -= c
    
env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=45, wsupport=[0,2,1000], expwsq=100)
happrox = MLE.MLE.Online.HistApprox(wmin=0, wmax=1000, numbuckets=10)
onlineci = MLE.MLE.Online.CI(wmin=0, wmax=1000, rmin=0, rmax=1, alpha=0.05)
_, samples = env.sample(1 << 16)
from pprint import pformat
print(pformat(samples), flush=True)

for n, (c, w, r) in enumerate(batchtoonline(samples)):
    happrox.update(c, w, r)
    onlineci.update(happrox.iterator)
    
    if n & (n - 1) == 0 and n & 0xAAAAAAAA == 0:
        from pprint import pformat
        print(pformat(
                {
                    'n': n,
                    'onlineci': onlineci.getqfunc(happrox.iterator),
                    'batchmle': MLE.MLE.estimate(happrox.iterator, wmin=0, wmax=1000), 
                    'batchci': MLE.MLE.asymptoticconfidenceinterval(happrox.iterator, wmin=0, wmax=1000)
                }), 
              flush=True)

[(35818, 0, 0), (5658, 2, 0), (24056, 2, 1), (2, 1000, 0), (2, 1000, 1)]
{'batchci': ((0, 1), (None, None)),
 'batchmle': (1.0,
              {'betastar': 1.0000000000000002,
               'num': 1.0000000000000002,
               'qfunc': <function estimate.<locals>.<lambda> at 0x7ff4db7ffb70>,
               'vmax': 1.0,
               'vmin': 0.9999999999999999}),
 'n': 0,
 'onlineci': {'betastar': 0.7090212316407815,
              'ci': False,
              'qfunc': <function Online.CI.getqfunc.<locals>.<lambda> at 0x7ff4dc769d08>}}
{'batchci': ((-1.000575304586373e-05, 1.0000100056782093),
             ({'betastar': 2.4394412698756992e-09,
               'gammastar': 1.0002833325839528e-05,
               'kappastar': -2.4013937716326954e-10,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7ff4db802ae8>},
              {'betastar': 1.0000000024750761,
               'gammastar': 1.0002833325839528e-05,
               'kappastar': -1.849036479

{'batchci': ((0.7157945447900277, 0.8126172713319192),
             ({'betastar': -0.02802318641243748,
               'gammastar': 64.63126478063161,
               'kappastar': 0.003986514259323193,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7ff4db7ffb70>},
              {'betastar': 0.7711863548622753,
               'gammastar': 228.8136625636666,
               'kappastar': 0.013962296713286639,
               'qfunc': <function asymptoticconfidenceinterval.<locals>.<lambda> at 0x7ff4db802ea0>})),
 'batchmle': (0.731469975688039,
              {'betastar': -5.824243854985619,
               'num': 16384.99999999967,
               'qfunc': <function estimate.<locals>.<lambda> at 0x7ff4db802d08>,
               'vmax': 0.7314699756880393,
               'vmin': 0.7314699756880386}),
 'n': 16384,
 'onlineci': {'betastar': -0.02123037235735025,
              'ci': True,
              'gammastar': 46.383554937965236,
              'kappastar'